# Programming Project 5 Solutions

The implementation of assignment tasks is done in Spark using the pyspark library as well as numpy and pandas to deal with the matrices conveniently.



In [1]:
# importing necessary packages

import numpy as np
import pandas as pd
import csv
import random

# For ALS

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# For SVD

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Matrix, Matrices


**NOTE: Make sure to rerun all the related cells instead of 1 cell. Beacause every cell is connected. That is, when rerunning a cell in Question - 1, rerun all the cells above that cell in this question.**  

### Question - 1

Consider a 25-by-20 Ratings matrix, Containing rows for 25 users and columns for 20 movies. Each cell of the table contains a rating value between 1(=Poor) and 7(=Best). Perform the following steps with this data structure:

#### Part - a

Randomly populate this table with 200 entries (between 1 and 7) and adjust the values to make sure that each row has at least 3 ratings and each column has at least 3 ratings. Leave the other 300 cells as unfilled. Include this table in your submission, shown in table form.


In [2]:
# Creating the entire array 

np.random.seed(73)

usr_mat = np.random.randint(1,7,500).reshape((25,20))

pd.DataFrame(usr_mat)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3,3,1,1,4,2,3,2,5,6,5,4,2,6,4,3,5,5,4,6
1,2,1,4,2,4,2,5,5,6,2,6,3,5,4,3,2,4,6,5,2
2,1,6,6,1,5,4,6,1,1,4,3,1,6,2,4,3,4,1,4,4
3,3,2,1,1,1,3,3,5,6,6,2,4,2,4,2,4,2,2,1,2
4,6,3,4,5,5,2,5,5,4,3,5,5,3,6,3,2,3,2,2,2
5,5,3,4,2,3,4,1,2,1,6,5,4,5,3,5,3,5,4,4,4
6,5,4,1,1,4,1,6,4,3,2,1,4,4,5,6,2,3,6,3,5
7,3,2,1,1,2,6,5,5,5,5,4,6,1,6,5,5,5,1,5,2
8,5,1,3,3,1,5,5,4,3,1,6,5,2,4,4,2,3,4,3,6
9,6,6,5,4,1,5,2,3,2,2,6,2,1,5,5,3,2,6,3,5


In [3]:
# create a mask array and multiply element wise to get the actual array

usr_mask = np.random.choice([0, 1], size=(25,20), p=[3.0/5, 2.0/5])

ratingss_obs = np.sum(usr_mask)


pd.DataFrame(usr_mask)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,0
1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0
3,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,0,1
5,0,0,0,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,1,1
6,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,1
7,0,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0
8,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,1,1,0,0,1
9,1,1,0,0,0,1,1,1,0,0,1,0,0,1,0,1,1,0,0,0


In [4]:
# Multiplying the usr_mat with mask element wise to get the 
# required usr_mat as per task and replace zeros with NAN

usr_mat = (usr_mat*usr_mask).astype('float')

pd.DataFrame(usr_mat)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,5.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,4.0,2.0,4.0,2.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,0.0
2,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,3.0,6.0,0.0,2.0,3.0,0.0,0.0,2.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,4.0,0.0,3.0,5.0,0.0,5.0,0.0,4.0,4.0
6,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,5.0,6.0,2.0,3.0,0.0,0.0,5.0
7,0.0,2.0,1.0,0.0,2.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0,3.0,0.0,6.0,0.0,2.0,0.0,4.0,2.0,3.0,0.0,0.0,6.0
9,6.0,6.0,0.0,0.0,0.0,5.0,2.0,3.0,0.0,0.0,6.0,0.0,0.0,5.0,0.0,3.0,2.0,0.0,0.0,0.0


In [5]:
# Making a list of tuples of (UID, MID, rating)
# which are observed (Unobserved are 0's)

usr_tup = []

for i in range(25):
    for j in range(20):
        if usr_mat[i,j] != 0:
            usr_tup.append((i,j,usr_mat[i,j]))

# This tuple list is used to convert to rdd

print(usr_tup[0:6])        


[(0, 1, 3.0), (0, 5, 2.0), (0, 8, 5.0), (0, 10, 5.0), (0, 12, 2.0), (0, 13, 6.0)]


#### Part - b:

Randomly select 150 of the 200 ratings as the training dataset and the remaining 50 as the test dataset. Show both sets of data in the form of tables.

In [6]:
# Selecting 150 ratings as train and rest as test(51)

train_usr = random.sample(usr_tup, 150)

test_usr = [k for k in usr_tup if k not in train_usr]

print("length of train:",len(train_usr))

print("length of test:",len(test_usr))

train_np = np.zeros((25,20))

test_np = np.zeros((25,20))

for i in train_usr:
    train_np[i[0],i[1]] = i[2]

for i in test_usr:
    test_np[i[0],i[1]] = i[2]

print("train data :")
display(pd.DataFrame(train_np))

print("test data :")
test_df = pd.DataFrame(test_np)
display(test_df)

length of train: 150
length of test: 51
train data :


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,4.0,2.0,4.0,2.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,0.0
2,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,3.0,5.0,0.0,5.0,0.0,4.0,0.0
6,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,5.0,6.0,0.0,3.0,0.0,0.0,0.0
7,0.0,2.0,1.0,0.0,2.0,6.0,5.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,3.0,0.0,0.0,6.0
9,0.0,6.0,0.0,0.0,0.0,5.0,2.0,3.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0


test data :


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
9,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Part - c:

Use any toolbox to find a matrix factorization of the training data and include only two latent factors. (Use ALS based method and not an SVD method). Show the latent factor matrices, properly labeled.


In [7]:
# Implementing ALS on train data using pyspark module

train_usr_rdd = sc.parallelize(train_usr)

train_usr_rdd = train_usr_rdd.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),\
                                    rating=float(p[2])))

train_usr_df = spark.createDataFrame(train_usr_rdd)

als = ALS(rank = 2, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model = als.fit(train_usr_df)

usr_factors = model.userFactors.orderBy("id").collect()

movie_factors = model.itemFactors.orderBy("id").collect()

In [8]:
# Displaying user x latent factors

usr_factors_df = pd.DataFrame(0, index=np.arange(len(usr_factors)), columns=["factor1", "factor2"]) 

for i in range(len(usr_factors)):
    usr_factors_df.iloc[[i],[0]] = usr_factors[i]['features'][0]
    usr_factors_df.iloc[[i],[1]] = usr_factors[i]['features'][1]

print("user x latent factor :")

display(usr_factors_df)

user x latent factor :


,factor1,factor2
0,2.009397,0.034767
1,1.758779,0.586869
2,2.158709,0.018595
3,0.206071,2.655388
4,2.345542,0.442305
5,1.458094,1.176728
6,2.016995,0.285821
7,0.671759,1.419603
8,0.415020,1.954535
9,0.465693,1.737599


In [9]:
# Displaying user x latent factors

movie_factors_df = pd.DataFrame(0, index=np.arange(len(movie_factors)), columns=["factor1", "factor2"]) 

for i in range(len(movie_factors)):
    movie_factors_df.iloc[[i],[0]] = movie_factors[i]['features'][0]
    movie_factors_df.iloc[[i],[1]] = movie_factors[i]['features'][1]

print("movie x latent factor :")

display(movie_factors_df)

movie x latent factor :


,factor1,factor2
0,0.979099,2.589075
1,1.347877,2.630673
2,2.334602,0.354017
3,0.575426,2.031554
4,2.141827,0.288467
5,0.876051,2.750279
6,2.037729,1.656495
7,1.661107,1.346591
8,2.746248,0.462264
9,1.791102,0.906388


#### Part - d:

Use the factorized Ratings matrix to predict the ratings for the 50 ratings of the test-dataset. Measure the error in prediction as the Mean-Square-Error between the actual and the predicted ratings. Submit the test data (in matrix form) with each cell showing the actual and the predicted ratings. Also, report the MSE value computed for all the predictions.


In [10]:
# Predicting test data using ALS Model using pyspark module

test_usr_rdd = sc.parallelize(test_usr)

test_usr_rdd = test_usr_rdd.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),\
                                    rating=float(p[2])))

test_usr_df = spark.createDataFrame(test_usr_rdd)

predictions = model.transform(test_usr_df)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)


In [11]:
# Displaying actual and predictions in a table separated by comma

predictions_rdd = predictions.collect()

test_df = test_df.astype(str)

for i in predictions_rdd:
    test_df.iloc[i["userId"],i["movieId"]] = test_df.iloc[i["userId"],i["movieId"]]+","+str(round(i["prediction"],2))

display(test_df)

print("RMSE#1 :", rmse)

print("MSE#1 :", rmse**2)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.0,2.7",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"4.0,2.42",0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.0,3.11",0.0,0.0,0.0,"2.0,6.27","3.0,3.93",0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"1.0,4.55",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"4.0,3.75"
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"2.0,5.32",0.0,0.0,0.0,"5.0,1.69"
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.0,2.5",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,"1.0,5.7",0.0,0.0,0.0,"5.0,5.74",0.0,0.0,0.0,0.0,"6.0,3.58",0.0,0.0,0.0,"4.0,2.25",0.0,0.0,0.0,0.0,0.0
9,"6.0,4.95",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.0,0.51",0.0,0.0,0.0,0.0,0.0,0.0


RMSE#1 : 2.423605695595047
MSE#1 : 5.873864567720752


### Question - 2

Repeat Q#1 with the following difference:

Populate the Ratings matrix in step 1.a in such a way (you are free to choose the rating values) that the MSE obtained in 1.d is minimized. Repeat all steps of Q#1 and submit the answers.


#### Part - a:

Randomly populate this table with 200 entries (between 1 and 7) and adjust the values to make sure that each row has at least 3 ratings and each column has at least 3 ratings. Leave the other 300 cells as unfilled. Include this table in your submission, shown in table form.

**Criteria for chosing the ratings:**

It is given that the MSE should decrease when the matrix is reconstructed from the matrices Q(users x factors) and P (items x factors). 

Keeping this in mind each row has a definite mean **$\mu$** which is randomly selected to be a value between [1,7]. While the standard deviation of each mean is kept **0.5-1**. By doing this, we are poulating the user ratings very close to his mean ratings. This in turn will be easy to predict his ratings for other movies. In other words, ALS makes a guess which falls very near to the mean. Furthermre, you can also reduce the choices for mean, that is, bringing all the row means close together.

![alt text](paint1.png)

In [12]:
# Create the matrix and then use mask as before <img src="paint1.png" style="width: 800px;"/>

np.random.seed(73)

usr_mat2 = np.zeros((25,20))
choices = [1,2,3,4,5,6,7]

for i in range(25):
    mu, sigma = random.choice(choices), 0.75 # Each row has mean of mu and sd of sigma
    usr_mat2[i] = np.random.normal(mu, sigma, 20)
    
usr_mat2 = np.around(usr_mat2, decimals=1)

# Making minor adjustments

usr_mat2[usr_mat2<1] = 1
usr_mat2[usr_mat2>7] = 7

pd.DataFrame(usr_mat2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3.4,4.6,4.8,3.2,1.9,3.0,3.3,1.9,2.5,3.8,4.2,1.8,3.2,2.8,3.2,2.1,3.4,2.7,3.0,2.6
1,1.0,1.0,1.0,1.0,1.4,1.8,1.1,1.0,1.9,2.2,1.0,1.0,1.0,1.8,1.0,1.0,1.7,1.7,1.7,1.0
2,5.5,6.1,6.4,4.8,7.0,5.2,6.1,5.6,6.5,6.2,6.5,6.8,5.7,6.3,6.0,4.7,6.0,6.6,6.4,5.2
3,2.8,1.5,1.5,2.3,1.8,3.1,2.1,2.7,2.0,2.1,2.4,1.0,1.6,1.0,1.0,2.6,2.6,2.1,1.8,1.9
4,5.1,3.6,4.3,4.0,6.0,4.6,3.9,5.3,3.9,3.8,3.3,3.6,3.2,5.6,3.7,3.8,3.7,3.2,3.5,4.1
5,5.7,6.5,5.5,5.1,5.2,6.7,5.6,4.2,7.0,4.6,5.6,5.8,4.5,4.9,6.5,6.2,6.1,6.1,6.0,5.5
6,4.5,4.9,4.4,4.1,5.4,6.7,4.5,3.9,5.0,5.0,5.6,4.9,4.9,5.4,5.4,5.7,4.5,4.2,4.1,4.8
7,7.0,6.3,5.9,7.0,7.0,6.0,6.3,7.0,7.0,6.7,6.8,6.4,5.8,6.7,6.7,6.6,6.4,7.0,6.9,7.0
8,5.1,5.5,4.9,4.6,5.2,4.1,5.7,5.9,3.9,6.5,6.3,5.1,4.1,6.3,4.7,4.8,4.9,5.7,5.2,5.0
9,2.8,1.7,2.7,1.9,2.3,1.4,2.7,2.2,1.8,1.0,2.5,2.0,1.5,2.1,2.1,2.3,1.6,3.0,1.9,2.0


In [13]:
# masking the above matrix

np.random.seed(156)

usr_mask2 = np.random.choice([0, 1], size=(25,20), p=[3.0/5, 2.0/5])

ratingss_obs2 = np.sum(usr_mask2)

print(ratingss_obs2)

pd.DataFrame(usr_mask2)


200


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,1
2,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0
3,0,1,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,1,0,0
4,0,0,0,1,0,0,1,0,1,1,0,1,1,0,1,0,1,0,0,0
5,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,1
6,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1
7,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0
8,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,1,0,0,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,0


In [14]:
# Multiplying the usr_mat with mask element wise to get the 
# required usr_mat as per task and replace zeros with NAN

usr_mat2 = (usr_mat2*usr_mask2).astype('float')

pd.DataFrame(usr_mat2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,3.0,3.3,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.7,0.0,1.0
2,0.0,6.1,0.0,0.0,7.0,0.0,0.0,5.6,0.0,6.2,6.5,0.0,0.0,0.0,0.0,0.0,0.0,6.6,0.0,0.0
3,0.0,1.5,1.5,0.0,1.8,0.0,2.1,2.7,0.0,2.1,0.0,1.0,0.0,0.0,1.0,0.0,2.6,2.1,0.0,0.0
4,0.0,0.0,0.0,4.0,0.0,0.0,3.9,0.0,3.9,3.8,0.0,3.6,3.2,0.0,3.7,0.0,3.7,0.0,0.0,0.0
5,5.7,6.5,0.0,0.0,5.2,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,6.2,6.1,6.1,0.0,5.5
6,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0,0.0,5.6,4.9,0.0,5.4,5.4,0.0,0.0,4.2,4.1,4.8
7,0.0,0.0,5.9,0.0,0.0,0.0,0.0,7.0,0.0,0.0,6.8,0.0,0.0,0.0,6.7,0.0,6.4,7.0,0.0,0.0
8,0.0,5.5,0.0,0.0,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.2,0.0
9,2.8,0.0,0.0,1.9,0.0,0.0,0.0,0.0,1.8,1.0,0.0,0.0,1.5,0.0,2.1,0.0,1.6,3.0,1.9,0.0


In [15]:
# Making a list of tuples of (UID, MID, rating)
# which are observed (Unobserved are 0's)

usr_tup2 = []

for i in range(25):
    for j in range(20):
        if usr_mat2[i,j] != 0:
            usr_tup2.append((i,j,usr_mat2[i,j]))

# This tuple list is used to convert to rdd

print(usr_tup2[0:6])   

print(len(usr_tup2))


[(0, 5, 3.0), (0, 6, 3.3), (0, 7, 1.9), (0, 15, 2.1), (1, 0, 1.0), (1, 8, 1.9)]
200


#### Part - b:

Randomly select 150 of the 200 ratings as the training dataset and the remaining 50 as the test dataset. Show both sets of data in the form of tables.

In [16]:
# Selecting 150 ratings as train and rest as test(51)

train_usr2 = random.sample(usr_tup2, 150)

test_usr2 = [k for k in usr_tup2 if k not in train_usr2]

print("length of train:",len(train_usr2))

print("length of test:",len(test_usr2))

train_np2 = np.zeros((25,20))

test_np2 = np.zeros((25,20))

for i in train_usr2:
    train_np2[i[0],i[1]] = i[2]

for i in test_usr2:
    test_np2[i[0],i[1]] = i[2]

print("train data :")
display(pd.DataFrame(train_np2))

print("test data :")
test_df2 = pd.DataFrame(test_np2)
display(test_df2)

length of train: 150
length of test: 50
train data :


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,3.3,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.7,0.0,1.0
2,0.0,6.1,0.0,0.0,7.0,0.0,0.0,0.0,0.0,6.2,6.5,0.0,0.0,0.0,0.0,0.0,0.0,6.6,0.0,0.0
3,0.0,1.5,1.5,0.0,0.0,0.0,0.0,2.7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.6,2.1,0.0,0.0
4,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.9,3.8,0.0,3.6,3.2,0.0,3.7,0.0,0.0,0.0,0.0,0.0
5,5.7,6.5,0.0,0.0,5.2,6.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,6.2,6.1,6.1,0.0,5.5
6,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0,0.0,5.6,4.9,0.0,5.4,0.0,0.0,0.0,4.2,4.1,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,6.8,0.0,0.0,0.0,6.7,0.0,6.4,7.0,0.0,0.0
8,0.0,5.5,0.0,0.0,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.1,0.0,1.6,3.0,1.9,0.0


test data :


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.8,0.0,2.1,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.4,0.0,0.0,0.0,0.0,4.8
7,0.0,0.0,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.2,0.0
9,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Part - c:

Use any toolbox to find a matrix factorization of the training data and include only two latent factors. (Use ALS based method and not an SVD method). Show the latent factor matrices, properly labeled.


In [17]:
# Implementing ALS on train data using pyspark module

train_usr_rdd2 = sc.parallelize(train_usr2)

train_usr_rdd2 = train_usr_rdd2.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),\
                                    rating=float(p[2])))

train_usr_df2 = spark.createDataFrame(train_usr_rdd2)

als2 = ALS(rank = 2, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model2 = als2.fit(train_usr_df2)

usr_factors2 = model2.userFactors.orderBy("id").collect()

movie_factors2 = model2.itemFactors.orderBy("id").collect()

In [18]:
# Displaying user x latent factors

usr_factors_df2 = pd.DataFrame(0, index=np.arange(len(usr_factors2)), columns=["factor1", "factor2"]) 

for i in range(len(usr_factors2)):
    usr_factors_df2.iloc[[i],[0]] = usr_factors2[i]['features'][0]
    usr_factors_df2.iloc[[i],[1]] = usr_factors2[i]['features'][1]

print("user x latent factor :")

display(usr_factors_df2)

user x latent factor :


,factor1,factor2
0,-0.929944,1.034420
1,0.215356,0.656841
2,-0.071750,3.046519
3,-0.647012,0.969518
4,0.613102,1.674519
5,0.639115,2.858371
6,0.808812,2.068292
7,0.165395,3.149588
8,0.312504,2.517972
9,-0.755979,1.077648


In [19]:
# Displaying user x latent factors

movie_factors_df2 = pd.DataFrame(0, index=np.arange(len(movie_factors2)), columns=["factor1", "factor2"]) 

for i in range(len(movie_factors2)):
    movie_factors_df2.iloc[[i],[0]] = movie_factors2[i]['features'][0]
    movie_factors_df2.iloc[[i],[1]] = movie_factors2[i]['features'][1]

print("movie x latent factor :")

display(movie_factors_df2)

movie x latent factor :


,factor1,factor2
0,-0.397018,1.939503
1,0.407217,2.052237
2,0.542156,2.064767
3,1.327684,1.783964
4,-0.542207,2.117001
5,0.022755,2.183808
6,-0.477288,2.218868
7,0.064453,2.116784
8,0.383966,2.090983
9,0.938459,1.968667


#### Part - d:

Use the factorized Ratings matrix to predict the ratings for the 50 ratings of the test-dataset. Measure the error in prediction as the Mean-Square-Error between the actual and the predicted ratings. Submit the test data (in matrix form) with each cell showing the actual and the predicted ratings. Also, report the MSE value computed for all the predictions.


In [20]:
# Predicting test data using ALS Model using pyspark module

test_usr_rdd2 = sc.parallelize(test_usr2)

test_usr_rdd2 = test_usr_rdd2.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),\
                                    rating=float(p[2])))

test_usr_df2 = spark.createDataFrame(test_usr_rdd2)

predictions2 = model2.transform(test_usr_df2)

evaluator2 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse2 = evaluator2.evaluate(predictions2)


In [21]:
# Displaying actual and predictions in a table separated by comma

predictions_rdd2 = predictions2.collect()

test_df2 = test_df2.astype(str)

for i in predictions_rdd2:
    test_df2.iloc[i["userId"],i["movieId"]] = test_df2.iloc[i["userId"],i["movieId"]]+","+str(round(i["prediction"],2))

display(test_df2)

print("RMSE#2 :", rmse2)

print("MSE#2 :", rmse2**2)

print("RMSE#1 :", rmse)

print("MSE#1 :", rmse**2)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,"3.0,2.24",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"1.0,1.57",0.0,0.0,0.0,0.0,"1.0,1.45",0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.6,6.44",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,"1.8,2.4",0.0,"2.1,2.46",0.0,0.0,"2.1,1.3",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,"3.9,3.42",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"3.7,3.47",0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.4,4.72",0.0,0.0,0.0,0.0,"4.8,4.08"
7,0.0,0.0,"5.9,6.59",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"5.2,5.09",0.0
9,0.0,0.0,0.0,"1.9,0.92",0.0,0.0,0.0,0.0,"1.8,1.96",0.0,0.0,0.0,"1.5,2.29",0.0,0.0,0.0,0.0,0.0,0.0,0.0


RMSE#2 : 0.6779609543190904
MSE#2 : 0.45963105558125184
RMSE#1 : 2.423605695595047
MSE#1 : 5.873864567720752


### Observations :

It is observed that there is a significant drop in MSE from around 8.5 to 0.5. Hence, the distribution of each row in the user-movie rating matrix matters.

### Question - 3	

Repeat Q#2 with the following difference:

#### Part -  a:

Now populate the Ratings matrix in such a way that the MSE obtained in 1.d is maximized.

**Criteria for chosing the ratings:**

It is given that the MSE should increase when the matrix is reconstructed from the matrices Q(users x factors) and P (items x factors). 

Keeping this in mind each row has a definite mean **$\mu$** which is randomly selected to be a value between [1,7]. While the standard deviation of each mean is kept as high as **2-4**. By doing this, we are poulating the user ratings very far to his mean ratings. This in turn will be hard to predict his ratings for other movies. In other words, ALS makes a guess which falls very far from the mean. As well increase the randomness in the ratings.

![alt text](paint2.png "Title")

In [22]:
# Create the matrix and then use mask as before

np.random.seed(75)

usr_mat3 = np.zeros((25,20))
choices = [1,2,3,4,5,6,7]

for i in range(25):
    mu, sigma = random.choice(choices), 3 # Each row has mean of mu and sd of sigma
    usr_mat3[i] = np.random.normal(mu, sigma, 20)
    
usr_mat3 = np.around(usr_mat3, decimals=1)

# Making minor adjustments to increase randomness

for i in range(25):
    for j in range(20):
        if usr_mat3[i,j] < 1 :
            usr_mat3[i,j] = random.choice(choices)
        if usr_mat3[i,j] > 7 :
            usr_mat3[i,j] = random.choice(choices)

pd.DataFrame(usr_mat3)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,5.0,2.3,3.4,1.0,3.4,7.0,3.8,1.5,6.0,5.5,2.6,3.5,1.0,1.0,3.8,7.0,5.0,2.0,4.1,5.1
1,6.9,1.7,6.6,6.0,4.1,6.5,4.6,4.0,1.1,3.9,5.1,5.0,1.0,4.5,3.3,4.0,5.0,4.0,4.0,5.0
2,2.0,5.0,6.8,2.0,5.0,1.0,2.6,3.0,6.0,4.4,6.0,6.0,5.9,4.4,5.0,4.8,2.1,5.0,2.0,2.0
3,3.0,1.5,3.0,7.0,6.7,3.3,2.0,3.0,7.0,5.0,2.3,7.0,3.0,4.6,1.0,3.7,2.0,7.0,1.0,4.0
4,3.0,2.8,3.0,4.3,3.0,4.6,2.3,5.0,5.1,5.0,6.0,5.0,7.0,3.9,6.0,3.3,2.6,2.6,3.7,6.5
5,6.8,2.0,6.3,4.0,5.0,6.0,4.3,5.0,5.2,3.5,3.0,3.0,7.0,5.7,2.0,3.0,4.8,3.0,5.7,6.7
6,2.2,3.9,1.4,1.0,1.0,4.2,6.4,4.9,3.5,6.0,1.8,2.1,3.0,4.0,4.0,3.5,2.0,7.0,2.9,4.5
7,2.0,7.0,4.0,2.8,2.0,2.4,4.5,1.0,4.4,1.2,4.7,1.0,1.0,6.0,5.5,2.9,7.0,4.7,4.7,2.0
8,7.0,1.3,5.0,5.0,1.6,2.0,7.0,4.0,3.3,2.0,6.0,3.0,7.0,6.0,2.8,3.0,7.0,5.7,2.8,4.0
9,5.7,6.5,5.2,5.0,1.2,4.8,4.4,5.0,6.6,2.2,1.7,4.7,4.3,2.5,5.6,1.1,4.8,7.0,3.0,1.0


In [23]:
# masking the above matrix

np.random.seed(59)

usr_mask3 = np.random.choice([0, 1], size=(25,20), p=[3.0/5, 2.0/5])

ratingss_obs3 = np.sum(usr_mask3)

print(ratingss_obs3)

pd.DataFrame(usr_mask3)


199


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0
2,0,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,1,1,0,1
3,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,1
4,0,1,0,0,0,1,0,1,1,1,1,0,0,1,0,0,0,1,1,1
5,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1
6,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,1,1,0,0,0
7,0,0,0,1,1,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0
8,1,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,0,1,1,1
9,1,0,1,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1


In [24]:
# Multiplying the usr_mat with mask element wise to get the 
# required usr_mat as per task and replace zeros with NAN

usr_mat3 = (usr_mat3*usr_mask3).astype('float')

pd.DataFrame(usr_mat3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,5.0,0.0,3.4,0.0,0.0,0.0,0.0,0.0,0.0,5.5,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.9,0.0,0.0,6.0,0.0,6.5,4.6,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0
2,0.0,5.0,6.8,2.0,0.0,0.0,2.6,3.0,6.0,0.0,0.0,0.0,0.0,4.4,5.0,0.0,2.1,5.0,0.0,2.0
3,0.0,1.5,0.0,0.0,6.7,0.0,2.0,0.0,0.0,5.0,0.0,7.0,0.0,4.6,0.0,3.7,0.0,7.0,1.0,4.0
4,0.0,2.8,0.0,0.0,0.0,4.6,0.0,5.0,5.1,5.0,6.0,0.0,0.0,3.9,0.0,0.0,0.0,2.6,3.7,6.5
5,0.0,0.0,0.0,0.0,0.0,6.0,4.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.7,6.7
6,2.2,3.9,1.4,1.0,1.0,0.0,0.0,4.9,0.0,6.0,1.8,0.0,3.0,0.0,4.0,3.5,2.0,0.0,0.0,0.0
7,0.0,0.0,0.0,2.8,2.0,0.0,0.0,0.0,0.0,0.0,4.7,0.0,1.0,6.0,5.5,0.0,0.0,0.0,0.0,0.0
8,7.0,0.0,0.0,0.0,1.6,0.0,7.0,4.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,5.7,2.8,4.0
9,5.7,0.0,5.2,0.0,0.0,4.8,4.4,5.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,4.8,0.0,0.0,1.0


In [25]:
# Making a list of tuples of (UID, MID, rating)
# which are observed (Unobserved are 0's)

usr_tup3 = []

for i in range(25):
    for j in range(20):
        if usr_mat3[i,j] != 0:
            usr_tup3.append((i,j,usr_mat3[i,j]))

# This tuple list is used to convert to rdd

print(usr_tup3[0:6])   

print(len(usr_tup3))


[(0, 0, 5.0), (0, 2, 3.4), (0, 9, 5.5), (0, 10, 2.6), (1, 0, 6.9), (1, 3, 6.0)]
199


#### Part - b:

Comment on the characteristics of the data distribution that increase or decrease the MSE. Show how the data in your tables in #2 and #3 above reflect those characteristics.

In [26]:
# Implementing ALS on the data usr_tup3

ratings_rdd = sc.parallelize(usr_tup3)

ratings_rdd = ratings_rdd.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2])))

ratings = spark.createDataFrame(ratings_rdd)

(training, test) = ratings.randomSplit([0.6, 0.4], seed = 69)

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

als3 = ALS(rank = 2, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model3 = als3.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions3 = model3.transform(test)

evaluator3 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse3 = evaluator3.evaluate(predictions3)


In [27]:
print("RMSE#3 :", rmse3)

print("MSE#3 :", rmse3**2)

print("\n")

print("RMSE#2 :", rmse2)

print("MSE#2 :", rmse2**2)

print("\n")

print("RMSE#1 :", rmse)

print("MSE#1 :", rmse**2)


RMSE#3 : 3.441610871198181
MSE#3 : 11.844685388749502


RMSE#2 : 0.6779609543190904
MSE#2 : 0.45963105558125184


RMSE#1 : 2.423605695595047
MSE#1 : 5.873864567720752


### Comments on characteristics of data:

As discussed in the criteria to minimize MSE in Question 2, the ratings in each row should be populated around the row mean as shown in the figure below:

![alt text](paint1.png "Title")

In the third question, the criteria is to maximize MSE so ratings in each row should be varied or random. This is done by increasing the sd as well as externally adding the randomnesss to the data.

![alt text](paint2.png "Title")


### Question - 4

Repeat #3 with the difference that now you use four latent factors. Use the same rating values as used in #3 above. Compare the predicted outcomes and the MSE values with those obtained in #3. Comment on any significant differences observed.

This is done by **increasing the rank from 2 to 4 while creating the ALS model**.

In [28]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

als4 = ALS(rank = 4, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

model4 = als4.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions4 = model4.transform(test)

evaluator4 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse4 = evaluator4.evaluate(predictions4)


In [29]:
print("RMSE#4 :", rmse4)

print("MSE#4 :", rmse4**2)

print("\n")

print("RMSE#3 :", rmse3)

print("MSE#3 :", rmse3**2)

print("\n")

print("RMSE#2 :", rmse2)

print("MSE#2 :", rmse2**2)

print("\n")

print("RMSE#1 :", rmse)

print("MSE#1 :", rmse**2)


RMSE#4 : 2.4953101951619026
MSE#4 : 6.226572970078932


RMSE#3 : 3.441610871198181
MSE#3 : 11.844685388749502


RMSE#2 : 0.6779609543190904
MSE#2 : 0.45963105558125184


RMSE#1 : 2.423605695595047
MSE#1 : 5.873864567720752


### Comparing MSE of #3 and #4:

There is a considerable drop in MSE from 3 to 4 as the latent factors increased from 2 to 4. This is beacause it possible to capture the more variance in the data with the help of more latent factors.

### Question - 5 

Perform the following steps with the “SVD-Data” matrix attached with this assignment. This table contains documents as rows and words as columns. Each cell entry is the number of times a word occurs in a document. Use any available toolbox to perform the SVD of this matrix. Then answer all the following questions.

#### Part - a	

Show all the Eigen-values and Eigen-vectors obtained after the decomposition.


In [30]:
# Importing the data 

rows = sc.textFile("SVD-Data.csv")

print(rows.take(3))


['6,12,3,4,8,2,2,0,0,0,0,0,1,2,3,10,0,2,7,6,12,0', '12,5,5,0,15,2,0,0,0,0,0,0,0,6,4,12,0,0,8,6,8,0', '10,8,4,6,12,4,9,0,0,0,0,0,8,8,5,16,0,4,5,2,4,0']


In [31]:
# Converting the each element in the rdd of type string
# separated by the "," to collection of integer elements 

rows = rows.map(lambda x: list(map(int, x.split(","))))

print(rows.take(3))


[[6, 12, 3, 4, 8, 2, 2, 0, 0, 0, 0, 0, 1, 2, 3, 10, 0, 2, 7, 6, 12, 0], [12, 5, 5, 0, 15, 2, 0, 0, 0, 0, 0, 0, 0, 6, 4, 12, 0, 0, 8, 6, 8, 0], [10, 8, 4, 6, 12, 4, 9, 0, 0, 0, 0, 0, 8, 8, 5, 16, 0, 4, 5, 2, 4, 0]]


In [32]:
# Converting the collection of integer to RowMatrix

mat = RowMatrix(rows)

print(mat.rows.take(3))


[DenseVector([6.0, 12.0, 3.0, 4.0, 8.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 10.0, 0.0, 2.0, 7.0, 6.0, 12.0, 0.0]), DenseVector([12.0, 5.0, 5.0, 0.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 4.0, 12.0, 0.0, 0.0, 8.0, 6.0, 8.0, 0.0]), DenseVector([10.0, 8.0, 4.0, 6.0, 12.0, 4.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 5.0, 16.0, 0.0, 4.0, 5.0, 2.0, 4.0, 0.0])]


In [33]:
# Compute the all singular values and corresponding singular vectors.
# The factor/Concept is given 22 the size of the 2nd dimension to get 
# all the eigen vectors an eigen values

svd = mat.computeSVD(22, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

U = U.rows.collect()

# Converting U to U_n aka a numpy array
U_n = np.array(U)

# Converting U_n to dataframe to display as a table
# Displaying the U matrix as a table below
# U_n is row indexed with doc number
pd.set_option('display.max_columns', 50)

df_U_n = pd.DataFrame(U_n)

df_U_n.index = range(1,len(df_U_n)+1)

print("U_n:")

df_U_n

U_n:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
1,-0.187704,-0.015574,0.142029,-0.263801,-0.154558,-0.094075,-0.291481,-0.514993,-0.155885,0.097620,0.044732,-0.369044,-0.082623,-0.066868,0.282071,0.161422,0.054637,0.170662,0.293221,0.001132,-0.038584,-0.097215
2,-0.194671,0.023286,0.199418,-0.371261,-0.171930,0.014002,-0.121972,0.024630,0.498755,-0.166668,0.016954,0.123962,0.166480,-0.329897,-0.206354,-0.212550,0.040724,0.320669,-0.055774,0.148004,-0.245006,0.153125
3,-0.244653,0.106584,0.175857,-0.269557,-0.056349,-0.447640,0.379093,0.026238,-0.165607,-0.043269,0.042821,-0.149933,0.128606,-0.045139,-0.247923,0.278204,-0.089184,-0.294031,-0.069330,-0.108623,-0.185247,-0.143868
4,-0.270461,0.063099,0.235495,-0.423599,0.021271,0.009072,-0.011216,0.275049,-0.191820,0.033262,-0.139383,-0.012936,-0.226788,0.186107,-0.002840,-0.295162,0.261520,-0.120771,-0.027759,-0.041589,0.471186,0.132694
5,-0.213939,0.028232,0.220951,-0.248926,0.199513,0.518639,-0.113460,0.141207,0.138466,0.110877,0.058708,0.262145,0.097861,0.177915,0.153239,0.296126,-0.248545,-0.244570,0.064836,-0.037788,-0.101918,-0.213561
6,-0.105239,-0.334669,0.180441,0.167611,0.057343,-0.343373,-0.178986,0.125103,0.278294,0.216639,-0.385281,-0.037477,0.042346,0.215189,0.310597,0.147626,0.225446,-0.083605,-0.231496,0.119325,-0.207913,-0.056250
7,-0.097663,-0.338889,0.195276,0.121336,0.132864,-0.313180,-0.217791,0.086820,-0.002465,-0.039843,0.563355,0.038388,0.200378,0.318219,-0.165179,-0.131480,-0.155802,0.135363,0.042361,0.082196,0.194931,0.010056
8,-0.158498,-0.337464,0.185587,0.133905,-0.147229,0.427666,0.219866,-0.101886,-0.126571,0.004956,0.185137,-0.312343,0.104741,0.007010,0.123724,-0.290880,0.103265,0.078151,-0.291579,-0.182492,-0.084031,-0.171839
9,-0.113897,-0.307382,0.150942,0.148783,-0.095362,0.193553,0.136761,-0.220523,-0.110098,-0.015942,0.115572,0.241455,-0.090208,0.001806,-0.299173,0.281186,0.279661,-0.184000,0.235239,0.087445,-0.163506,0.284308
10,-0.113170,-0.317670,0.168613,0.163098,0.028452,-0.088984,0.102553,0.060064,-0.117916,0.000145,-0.217869,0.133248,-0.280037,-0.423110,-0.206444,-0.177291,-0.129172,-0.071211,0.070636,0.078964,0.041726,-0.296645


In [34]:
# Printing out the singular values
print("Singular Values:")
print(s)

print('\n')

# s as a numpy array

s = np.array(s)

# Printing out the eigen values = squares of singular values
print("Eigen Values:")
print(s**2)


# Obtain the diagonal matrix of the singular values as numpy array

s_n = np.diag(s) 

# Converting the Dense Matrix of V as numpy array

V_n = V.toArray()

# This is the V matrix shown in the table below 

words = ["data", "program", "complexity", "concept", "algorithm", "processor", "game", "gene" , "drug", \
"disease", "election", "vote", "candidate", "optimal", "result", "training", "campaign", "win",\
"run", "best", "expression", "sport"]

print("V_n\n")
pd.DataFrame(V_n, index = words)

Singular Values:
[97.11456238982272,58.24921196151485,51.3044132324946,41.70461245520586,31.5948147432476,18.230618512690146,15.19893072863372,13.68517289044494,10.862144359419108,10.45294018607867,10.009688453555572,8.881503732094819,8.207839689275737,7.372291162982129,6.420087471207991,5.771771966900108,4.200389984195362,3.314536468985021,2.5909990750990595,2.36170825078564,1.8819490436488646,1.2070383791432648]


Eigen Values:
[9.43123823e+03 3.39297069e+03 2.63214282e+03 1.73927470e+03
 9.98232319e+02 3.32355451e+02 2.31007495e+02 1.87283957e+02
 1.17986180e+02 1.09263959e+02 1.00193863e+02 7.88811085e+01
 6.73686324e+01 5.43506770e+01 4.12175231e+01 3.33133516e+01
 1.76432760e+01 1.09861520e+01 6.71327621e+00 5.57766586e+00
 3.54173220e+00 1.45694165e+00]
V_n



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
data,-0.246198,0.094727,0.313341,-0.328581,0.476830,0.130766,0.040865,0.063721,0.206993,-0.032168,0.167776,0.364016,-0.061362,-0.103661,0.251865,-0.077903,-0.382596,0.048180,-0.032737,0.158554,0.102941,-0.002588
program,-0.188781,-0.010362,-0.020111,-0.337979,0.003311,-0.169183,-0.253057,-0.265646,-0.653820,0.095257,-0.153070,-0.012137,-0.212341,-0.082368,0.221322,-0.076899,0.162576,0.094380,-0.047390,0.134905,0.174352,0.182474
complexity,-0.104565,-0.036847,0.124432,-0.087943,0.009329,-0.004515,-0.021837,0.269044,0.084445,0.072769,-0.444959,-0.239913,-0.139494,-0.183457,-0.026901,-0.014795,-0.210961,0.326953,0.357496,-0.509110,0.106537,0.140503
concept,-0.095384,0.100234,0.090992,0.060213,0.236532,-0.192883,0.185568,-0.494610,-0.051892,-0.526105,-0.149298,-0.284998,-0.056222,-0.005155,0.075892,0.028726,-0.165932,-0.168478,-0.023808,-0.208288,-0.264098,-0.185318
algorithm,-0.160539,0.088149,0.258916,-0.428762,0.050795,-0.029862,-0.035623,0.045183,0.229079,-0.090727,-0.062191,-0.060552,0.142943,-0.257033,-0.398144,-0.041145,0.527208,-0.176915,-0.087933,-0.009958,-0.202690,0.188903
processor,-0.043473,0.016022,0.071789,-0.141127,0.001497,0.009779,0.011075,0.097868,-0.052820,0.031753,-0.030651,-0.013391,-0.034984,0.115868,-0.038059,0.073478,0.097354,-0.572135,0.112066,-0.227336,0.590490,-0.432244
game,-0.328617,0.381757,0.156474,0.326950,0.291905,0.019156,0.057349,-0.000782,-0.282890,0.322827,-0.066160,-0.032841,0.356755,0.037772,-0.250242,0.283623,0.037026,0.186130,-0.063574,0.009640,0.014134,-0.155606
gene,-0.091724,-0.409019,0.256851,0.251547,-0.028043,0.113972,0.267055,-0.087256,-0.240913,-0.041126,0.201238,0.072570,-0.192278,-0.312457,-0.405965,-0.387293,-0.018388,0.110895,0.007786,0.019904,0.117087,-0.143650
drug,-0.136625,-0.337754,0.018411,0.180609,0.141807,-0.351184,-0.530655,0.328593,-0.138912,-0.372346,0.104011,0.151415,0.209835,0.001641,-0.122868,0.139605,-0.120262,-0.090030,0.032456,-0.041216,-0.037362,0.065355
disease,-0.085042,-0.357049,0.206539,0.228792,0.023778,-0.140723,0.112957,0.114962,0.195212,0.113578,-0.528649,-0.097713,0.057923,-0.134469,0.340806,0.029721,0.138803,-0.078423,-0.286412,0.347126,0.040937,-0.125777


In [35]:
# Get back the original Matrix read from the data file

A = np.matmul(np.matmul(U_n, s_n), V_n.T)

df_A = pd.DataFrame(np.round(A,2), columns = words)

df_A.index = range(1,len(df_A)+1)

df_A

,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
1,6.0,12.0,3.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,10.0,0.0,2.0,7.0,6.0,12.0,0.0
2,12.0,5.0,5.0,0.0,15.0,2.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.0,4.0,12.0,0.0,0.0,8.0,6.0,8.0,0.0
3,10.0,8.0,4.0,6.0,12.0,4.0,9.0,-0.0,0.0,0.0,0.0,0.0,8.0,8.0,5.0,16.0,0.0,4.0,5.0,2.0,4.0,0.0
4,16.0,12.0,7.0,1.0,15.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.0,12.0,6.0,9.0,0.0,5.0,8.0,4.0,8.0,0.0
5,18.0,4.0,4.0,-0.0,10.0,4.0,8.0,0.0,0.0,-0.0,0.0,0.0,0.0,6.0,8.0,0.0,0.0,4.0,6.0,6.0,10.0,0.0
6,1.0,0.0,4.0,-0.0,0.0,-0.0,-0.0,8.0,12.0,16.0,0.0,0.0,8.0,-0.0,-0.0,0.0,0.0,1.0,0.0,4.0,12.0,2.0
7,4.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,12.0,15.0,8.0,1.0,0.0,8.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0
8,1.0,0.0,2.0,-0.0,0.0,-0.0,0.0,16.0,4.0,10.0,0.0,0.0,4.0,2.0,12.0,0.0,0.0,1.0,6.0,5.0,16.0,-0.0
9,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,14.0,6.0,8.0,0.0,0.0,5.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,12.0,-0.0
10,2.0,1.0,4.0,1.0,-0.0,-0.0,-0.0,15.0,9.0,12.0,0.0,0.0,6.0,0.0,4.0,0.0,0.0,2.0,0.0,5.0,8.0,-0.0


In [36]:
# Similar to the Original Data

df_original = pd.DataFrame(np.array(rows.collect()), columns = words)

df_original.index = range(1,len(df_original)+1)

df_original

,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
1,6,12,3,4,8,2,2,0,0,0,0,0,1,2,3,10,0,2,7,6,12,0
2,12,5,5,0,15,2,0,0,0,0,0,0,0,6,4,12,0,0,8,6,8,0
3,10,8,4,6,12,4,9,0,0,0,0,0,8,8,5,16,0,4,5,2,4,0
4,16,12,7,1,15,6,6,0,0,0,0,0,4,12,6,9,0,5,8,4,8,0
5,18,4,4,0,10,4,8,0,0,0,0,0,0,6,8,0,0,4,6,6,10,0
6,1,0,4,0,0,0,0,8,12,16,0,0,8,0,0,0,0,1,0,4,12,2
7,4,0,0,0,0,0,0,12,15,8,1,0,8,0,0,0,0,0,0,0,14,1
8,1,0,2,0,0,0,0,16,4,10,0,0,4,2,12,0,0,1,6,5,16,0
9,0,0,0,0,0,0,0,14,6,8,0,0,5,0,8,0,0,0,0,8,12,0
10,2,1,4,1,0,0,0,15,9,12,0,0,6,0,4,0,0,2,0,5,8,0


#### Part - b 

How many, and which Eigen values (and vectors) would you select if you should preserve 85% of the information?
    
    Eigen Values are the squares of the singular values. Retain only those eigen values whose sum of the squares is around 
    85% of the all eigen values sum of the squares.Summing is done such that all the higher eigen values are given priority.
    

In [37]:
# Singular values

print(s)

[97.11456239 58.24921196 51.30441323 41.70461246 31.59481474 18.23061851
 15.19893073 13.68517289 10.86214436 10.45294019 10.00968845  8.88150373
  8.20783969  7.37229116  6.42008747  5.77177197  4.20038998  3.31453647
  2.59099908  2.36170825  1.88194904  1.20703838]


In [38]:
# Since the singular values are already sorted we can
# directly square the singular values and produce cumulative sum
# Then divide by the sum of squares of all singular values
# We will get new np array : 
# e_var_frac => cumulative variance fraction contributed by each eigen value 

# np.sort(s)

e_s = np.sum(s**2)

e_var_frac = np.cumsum(s**2)/e_s

print("eigen values sum:",e_s)

print("cumulative eigen value contribution:\n",e_var_frac)

eigen values sum: 19593.000000000004
cumulative eigen value contribution:
 [0.48135754 0.65453013 0.78887111 0.87764132 0.92858974 0.94555271
 0.95734302 0.96690173 0.97292359 0.97850027 0.98361403 0.98764001
 0.99107842 0.9938524  0.99595609 0.99765635 0.99855684 0.99911756
 0.9994602  0.99974487 0.99992564 1.        ]


#### Selecting the Eigen Values and Eigen Vectors

From the above np array e_var_frac, cumulative variance fraction contributed by each eigen value, we can include **highest 4 singular values** and discard other eigen values and corresponding eigen vectors since the variance contributed by these is little above 85%.

#### Part - c

Take one of the dropped Eigen-vectors and interpret it in terms of documents and words to show its relevance/irrelevance to the original data table.

Lets take the 7th eigen value and its corresponding vectors in U and V, which are at 6th column respectively.

In [39]:
# pulling out the 7th eigen vector in U and V 

U_7ev = U_n[:,6].reshape((U_n[:,6].shape[0],1))

pd.DataFrame(U_7ev.T, columns = range(1, len(df_U_n)+1), index = [6])

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
6,-0.291481,-0.121972,0.379093,-0.011216,-0.11346,-0.178986,-0.217791,0.219866,0.136761,0.102553,0.113354,0.453068,0.110383,-0.18162,-0.245339,-0.302074,0.218421,-0.006242,-0.130933,-0.211707,-0.069106,0.1696,-0.102594,0.00691,-0.003278,0.133632


From the above we can see that the 7th eigen vector/6th concept will capture the following documents variance very well:

doc12, doc16, doc3, doc15 and doc1.

Similarly, lets take 6th vector in V. 

In [40]:
V_7ev = V_n[:,6].reshape((V_n[:,6].shape[0],1))

pd.DataFrame(V_7ev.T, columns = words, index = [6])

,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
6,0.040865,-0.253057,-0.021837,0.185568,-0.035623,0.011075,0.057349,0.267055,-0.530655,0.112957,0.088147,0.093846,0.450006,0.157961,0.315557,0.154704,0.00414,-0.103435,-0.204159,-0.181699,-0.225065,-0.159428


From the above we can see that the 7th eigen vecot/6th concept will capture the following words variance very well:

game, result, drug, candidate and program.

Yes, we will be losing information that this eigen value contributes, that is:

    eigen value = 231.007495/19593.000000000004 = 1.17% (approx)

Since, A reconstruted by using the below formula:

![alt text](cap2.png)

Lets assume that the green color stripes are the 7th singular value (square root of eigen value) and corresponding eigen vectors. The red color stripes are the eigen values greater than the chosen. When the red stripes are removed the corresponding term in the summation is removed. Since, the $\sigma$ very less the lost information is very less as computed above.

![alt text](cap1.png)

#### Part - d

How many concept vectors are chosen by you in step-b above? Write the intuitive interpretation of each of these vectors. What is the intuitive interpretation of the Eigen-values associated with each of these vectors?

The higher four singular values and corresponding concept vectors are chosen in the step-b. 

The intuitive interpretation:

These four concept vectors capture around the 85% (precisely 87.7%) of the variance of the documents. In other words, each row/document in **U** matrix will define how close it is with all columns/concepts chosen. 

Hence, the U matrix does the mapping from the document to concept space.

So, the linear combination of these concept vectors can express any document in the concept space. 

From careful observation of V_n (mapping from the words to concepts) it is revealed that the concept space mainly comprised of following four hypothetical concepts: 

1. Concept#0 (concept at column 0): Computer Science (CS) Concept
2. Concept#1 (concept at column 1): Genes Concept
3. Concept#2 (concept at column 2): Election Concept
4. Concept#3 (concept at column 3): Games Concept

Because some documents are best described as a vector in any combination of the above concepts space. For example: doc26 can be best expressed in concept formed by the CS and Games concept. While certain documents like doc6 - doc11 is most aligned to the Genes Concept (Concept in column 1 in U and V). This observation is based on the documents, its word occurences and the eigen vectors.

The high four singular values are correspond to the concept strength.

The visualization of these concepts is provided in part - f.

#### Part - e

Compute the MSE between the original matrix and the predicted matrix using the truncated number of Eigen vectors.


In [41]:
# Using the truncated eigen vectors to predict the matrix
# Considering the highest 4 singular values and corresponding vectors

trunc_A = np.matmul(np.matmul(U_n[:,0:4], s_n[0:4,0:4]), V_n[:,0:4].T)

df_trunc_A = pd.DataFrame(np.array(trunc_A), columns = words)

df_trunc_A.index = range(1,len(df_trunc_A)+1)

df_trunc_A


,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
1,10.300120,7.022448,3.813746,1.648384,9.450226,2.853672,3.187141,1.147217,0.944053,0.862010,0.992443,0.879097,3.453027,5.450831,4.652360,6.739441,0.880309,2.437138,5.762861,4.196929,7.446118,-0.758256
2,13.076268,8.582204,4.561587,1.937866,12.442253,3.763198,3.269074,-0.087635,-0.483232,-0.305883,0.129673,0.103457,2.911816,6.852879,4.126902,7.963891,-0.011752,1.444770,5.817767,3.715825,8.104615,-1.763957
3,12.958480,8.039034,4.366933,3.032609,11.517628,3.366585,7.914111,-0.870532,-0.715058,-0.904753,0.115780,0.133674,2.376344,7.345221,5.772223,9.998419,0.183787,4.862840,7.283962,4.876566,6.533272,2.277354
4,16.405203,10.648158,5.668035,2.909362,15.243391,4.561246,6.149106,-0.434719,-0.621048,-0.625079,0.328042,0.297983,3.570824,8.846655,6.055297,10.972770,0.238122,3.464103,8.145103,5.324335,9.548705,-0.453435
5,12.234016,7.185908,4.435423,2.552957,10.866573,3.208450,5.834920,1.533301,0.616918,1.145846,-0.593827,-0.547261,2.706234,6.619606,5.067154,8.254079,-0.602778,2.907943,5.660961,4.290761,8.417171,0.975983
6,1.273464,-0.417309,2.324172,0.284109,-0.677866,-0.189951,-0.349529,13.047064,9.413519,11.340862,-0.054371,-0.193512,5.506069,0.019878,4.842404,-0.777998,-0.189492,0.387930,0.667979,4.228740,12.265189,0.571664
7,1.941640,0.083289,2.520716,0.142339,0.206869,0.101122,-1.197036,12.790137,9.061465,11.081701,-0.204030,-0.334499,5.471436,0.261492,4.408933,-0.844391,-0.385219,-0.445992,0.455010,3.901626,12.616995,-0.429765
8,3.076034,1.030567,3.027475,0.700518,0.809186,0.249627,0.869710,13.302294,9.926140,11.571774,0.709134,0.498571,6.566350,1.070123,6.418701,0.893170,0.570378,1.819958,2.520896,5.646614,13.573200,1.005491
9,1.414800,0.020756,2.234253,0.338633,-0.457975,-0.125770,0.040011,11.887834,8.821873,10.352626,0.479346,0.304344,5.473241,0.206598,5.030344,-0.239313,0.355300,1.055428,1.375648,4.428611,11.348171,0.727917
10,1.428594,-0.206353,2.309270,0.390279,-0.543345,-0.157602,0.125102,12.509503,9.139144,10.884427,0.155298,0.005007,5.470126,0.181978,5.042708,-0.369075,0.036895,0.876829,1.085541,4.401187,11.843906,0.871891


In [42]:
# Obtaing MSE

nrow, ncol = A.shape

trunc_A_MSE = np.sum((A - trunc_A)**2)/(nrow*ncol)

print(trunc_A_MSE)

4.191212518402572


#### Part - f

The power-point slides set (Faloutsos-p1-SVD) shows four interpretations of an SVD. Briefly explain the results obtained for SVD of your data matrix, as per each of the four interpretations. 

**SVD Interpretation - 1:**

U: document-to-concept similarity matrix, each column is the eigen vector of $AA^T$ (document-document similarity matrix)

V: term-to-concept similarity matrix, each column is the eigen vector of $A^TA$ (term-term similarity matrix)

s: its diagonal elements: ‘strength’ of each concept, each diagonal element is eigen value of $AA^T$ or $A^TA$

#### Visualizing the concepts:

Note: This is the visualization showing the matrices U, s, V after reducing the concepts by taking only first 4 eigen values which capture 85% of variance. Same explanation is valid if more concepts are taken into account and but what each concept imples may change. 

![alt text](concepts_viz.png)

**SVD Interpretation - 2:**

Spectral Decomposition/ Dimensionality reduction:

The SVD will decompose the original matrix into U, S and V as shown below:

![alt text](cap2.png)

where each eigen vector will be the axis along which it can capture the maximum variance of the documents in terms space and vice-versa.

Thus, it enables us to chose the only axis along which maximum variance is captured and ignore the other since the variance offered by that dimension is very low.  

![alt text](cap3.png)


**SVD Interpretation - 3:**

Picking non-zero and rectangular ‘blobs’:

It is quite difficult to visualize rectangular blobs for each concept separately since many documents have equal variance in more than two concept dimensions. That is, a document can be associated with more than two concepts like CS and Game concept.
However, documents of Gene concept stands out as a rectangular blobs as shown in the figure below:

![alt text](cap4.png)

This blob made of doc6-doc11 and words like gene, drug and disease. 

**SVD Interpretation - 4:**

Fixed points - graph interpretation:

This is entirely a different perspective of eigen vectors. Here each eigen vector is a vector upon a given transformation A will result in scaling of the vector but no change in the direction. This is illustrated in the below picture.

![alt text](cap5.png)

It is intuitive that we can obtain the one which scales the vector in right direction as well as left. 

Fixed Point:
                                                
$ A^TA V_i = \lambda_i^2 V_i$
                                            
Convergence:
                                                    
$ A^TA V_i \approx (constant) V_i$                                                

$ V_i $ = right eigen vectors

$ U_i $ = left eigen vectors


#### Part - g

Consider using only the top two Eigen-values and Eigen vectors. Show and interpret how doc-1, doc-8, and doc-16 are summarized in this case?

In [43]:
# Considering top 2 Eigen-values and corresponding Eigen vectors

top2_A = np.matmul(np.matmul(U_n[:,0:2], s_n[0:2,0:2]), V_n[:,0:2].T)

pd.DataFrame(top2_A[[0,7,15],:], index=[1,8,16], columns = words )

,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
1,4.401943,3.450646,1.939520,1.647805,2.846464,0.777928,5.643962,2.043064,2.796904,1.874120,2.575852,2.338653,3.609867,2.829796,6.055065,5.359726,2.657387,6.188463,6.206104,5.295130,4.053590,3.521355
8,1.927526,3.109486,2.333821,-0.502113,0.738341,0.354210,-2.445992,9.451949,8.742237,8.327540,4.825426,4.244648,8.099349,0.475020,6.596713,0.701204,4.510113,2.751145,5.242486,6.384391,10.538674,-2.030290
16,3.649488,3.461054,2.165438,0.906645,2.169114,0.650047,2.832699,4.907153,5.141682,4.372645,3.523979,3.148243,5.440599,2.059158,6.501518,3.825575,3.459768,5.146147,6.093385,5.916710,6.651493,1.578035


In [44]:
# Actual doc-1, doc-8 and doc-16

df_original.iloc[[0,7,15],:]

,data,program,complexity,concept,algorithm,processor,game,gene,drug,disease,election,vote,candidate,optimal,result,training,campaign,win,run,best,expression,sport
1,6,12,3,4,8,2,2,0,0,0,0,0,1,2,3,10,0,2,7,6,12,0
8,1,0,2,0,0,0,0,16,4,10,0,0,4,2,12,0,0,1,6,5,16,0
16,0,5,1,1,0,0,0,0,6,0,8,9,4,0,5,0,12,9,10,9,3,0


**Interpretation:**

Upon observing the top2_A and original matrices, it is evident that decomposition of original matrix has no concept relating to Election and Game. Thus the words like:

game, sports, win, vote, candidate, result, run, election, etc., are incorrectly estimated.

For example:

    game is incorrectly estimated in all the three docs considered in the as : 5.64, -2.445 and 2.832 in doc1, doc8 and doc16 respectively.
    
Additionally, the concepts which are considered alone cannot estimate accurately the word count of their concept in a document. This is because the concept alone cannot capture the data variance entirely.  

#### Part - h

Compute the MSE if we use only the top two Eigen-vectors and compare it to the value obtained in #5d.


In [45]:
# Obtaing MSE

top2_A_MSE = np.sum((A - top2_A)**2)/(nrow*ncol)

print(top2_A_MSE)

11.833550835132431


**Comparision with 5d**

Yes, the mean square error has increased significantly. This is mainly due to ignoring the concepts variances provided by the other concepts.

                                                                                                
                                                                                                    Author:
                                                                                                    
                                                                                                    Varun Raj Rayabarapu
                                                                                                    University of Cincinnati
                                                                                                    rayabavj@mail.uc.edu